### **Computer Assignment 02: Naive Bayes**



In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
#Importing the needed packages for this code
import os
import numpy as np
from collections import Counter
import zipfile
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

This code is a Python function named "make_Dictionary" that takes the root directory as input and creates a dictionary of the 3000 most common words in the emails stored in the directory. 


In [31]:
def make_Dictionary(root_dir):
  all_words = [] #This first steps creates an empty list to store the words extracted
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)] #creates "emails" list
  for mail in emails:
    with open(mail) as m: 
      for line in m:
        words = line.split() #splits current line into words
        all_words += words
  dictionary = Counter(all_words) #creates "dictionary" with count of each word
  list_to_remove = list(dictionary)

  for item in list_to_remove:
    if item.isalpha() == False: #checks if alphabetic or not, returns True if items are alphabetic
      del dictionary[item] #removes item from "dictionary" if not alphabetical
    elif len(item) == 1:
      del dictionary[item] #removes single character and stop words
  dictionary = dictionary.most_common(3000) #returns the most 3000 most common words
  return dictionary 
            

This next function first returns "features matrix" and "train labels" and uses them to open and close files to make sure all words are added and counted in the dictionary properly. Then, the file path is split in order to identify spam vs real emails using specific naming. 



In [32]:
def extract_features(mail_dir):
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)] #create list of "files" to each directory by joining directory path "mail dir" with file name "fi"
  features_matrix = np.zeros((len(files),3000))
  train_labels = np.zeros(len(files))
  count = 1;
  docID = 0;
  for fil in files: #takes each file in "files" which contains the email files in directories
    with open(fil) as fi: #ensures file will be closed automatically after it's done processing the file
      for i, line in enumerate(fi):
        if i ==2: #if the line numer is 2, code splits line into words 
          words = line.split() #the resulting words from above are stored in the "words" variable
          for word in words:
            wordID = 0
            for i, d in enumerate(dictionary): #checks each word to see if it exists in the dictionary
              if d[0] == word:
                wordID = i
                features_matrix[docID,wordID] = words.count(word) #updates the entry with the count of that word in the current email
      train_labels[docID] = 0;
      filepathTokens = fil.split('/') #splits the file path for the current file into tokens 
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"): #identifies spam vs real mail based on naming 
        train_labels[docID] = 1;
        count = count + 1
      docID = docID + 1
  return features_matrix, train_labels                

This section first uploads the test and train data from our google drive. Then this code enters our data into the Gaussian Naive Bayes Classifier model and predicting which category our emails fall into and print an accuracy score for the algorithm. 

In [33]:
train_dir = '/content/drive/MyDrive/BSAN6070/CA02/train-mails' #importing data from my google drive
test_dir = '/content/drive/MyDrive/BSAN6070/CA02/test-mails' #importing data from my google 


In [34]:
dictionary=make_Dictionary(train_dir)

print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(train_dir) #processing and training of the train_dir folder
test_features_matrix, test_labels = extract_features(test_dir) #tests the trained model using the test_dir

model = GaussianNB() #imports the Gaussian Naive Bayes classifier to run the model

print ("Training Model using Gaussian Naibe Bayes algorithm .....")
model.fit(features_matrix, labels) #enters each function into the Gaussian Naive Bayes Classifier model 
print ("Training completed")
print ("testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features_matrix) #predicts which category to fall into
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels)) #shows ending accuracy score of the algorithms ability to predict spam vs real emails

reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9615384615384616


======================= END OF PROGRAM =========================